## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-18-04-03-19 +0000,missionlocal,RV dwellers say S.F. homeless outreach worker ...,https://missionlocal.org/2025/12/sf-rv-dweller...
1,2025-12-18-04-03-01 +0000,startribune,Trump gives a partisan prime-time address insi...,https://www.startribune.com/trump-is-previewin...
2,2025-12-18-04-02-03 +0000,bbc,Venezuela denounces Trump's order for ship blo...,https://www.bbc.com/news/articles/c4gej5ezyypo...
3,2025-12-18-04-00-00 +0000,wsj,Islamic State’s Caliphate Is Gone. Its Influen...,https://www.wsj.com/world/islamic-states-calip...
4,2025-12-18-03-59-10 +0000,nyt,Australia to Crack Down on Hate Speech After B...,https://www.nytimes.com/2025/12/17/us/australi...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2346/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
10,trump,61
42,reiner,22
36,bondi,21
155,new,17
113,oil,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
161,2025-12-17-21-58-00 +0000,wsj,President Trump on Tuesday received and discus...,https://www.wsj.com/politics/elections/trump-t...,157
203,2025-12-17-20-04-00 +0000,wsj,"Rep. Dan Newhouse, one of 10 House Republicans...",https://www.wsj.com/politics/elections/dan-new...,145
185,2025-12-17-20-48-00 +0000,wsj,The Senate passed the sweeping defense-policy ...,https://www.wsj.com/politics/national-security...,116
172,2025-12-17-21-29-04 +0000,nypost,House crushes Dems’ attempt to rein in Trump’s...,https://nypost.com/2025/12/17/world-news/house...,113
355,2025-12-17-11-00-00 +0000,latimes,Arellano: With his un-Christian Rob Reiner ins...,https://www.latimes.com/california/story/2025-...,112


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
161,157,2025-12-17-21-58-00 +0000,wsj,President Trump on Tuesday received and discus...,https://www.wsj.com/politics/elections/trump-t...
80,98,2025-12-18-00-39-21 +0000,nypost,Nick Reiner was ‘declining’ before parents’ mu...,https://nypost.com/2025/12/17/us-news/nick-rei...
158,69,2025-12-17-21-59-17 +0000,nypost,Fashion designer recalls hiding under a van fo...,https://nypost.com/2025/12/17/world-news/fashi...
203,66,2025-12-17-20-04-00 +0000,wsj,"Rep. Dan Newhouse, one of 10 House Republicans...",https://www.wsj.com/politics/elections/dan-new...
329,62,2025-12-17-12-11-13 +0000,latimes,"Warner Bros. rejects Paramount's hostile bid, ...",https://www.latimes.com/entertainment-arts/bus...
385,53,2025-12-17-08-36-35 +0000,wsj,"Stock Market News, Dec. 17, 2025: Tech Slide W...",https://www.wsj.com/livecoverage/stock-market-...
191,53,2025-12-17-20-36-00 +0000,wsj,FCC Chair Brendan Carr told a Senate oversight...,https://www.wsj.com/politics/policy/fcc-isnt-a...
124,43,2025-12-17-23-06-12 +0000,nypost,Senate approves bill inspired by DC plane cras...,https://nypost.com/2025/12/17/us-news/senate-a...
89,40,2025-12-18-00-21-00 +0000,wsj,Dan Bongino said that he is stepping down as d...,https://www.wsj.com/politics/policy/fbi-deputy...
85,38,2025-12-18-00-28-09 +0000,nypost,Missing Kentucky girl found alive with new ide...,https://nypost.com/2025/12/17/us-news/missing-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
